In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../source')
sys.path.append('../source/alpha_scaling')

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import Electrostatics
import Polarization

from Utilities import write_xyz
from Constants import KC
from AlphaHR import Alpha
#from AlphaR import Alpha

2023-06-30 08:51:37.866165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 08:51:39.715740: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
def get_batch(key):
    multipoles = MULTIPOLES[key]
    elements_1, elements_2 = DATA[key]['elements']
    distance_matrices = DATA[key]['distance_matrices']
    coords_1, coords_2 = DATA[key]['coordinates']
    graph_1, graph_2 = DATA[key]['graphs']
    alphas_1, alphas_2 = ALPHAS[key]
    return coords_1, coords_2, distance_matrices, elements_1, elements_2, graph_1, graph_2, alphas_1, alphas_2, multipoles

In [3]:
RATIOS = np.load('RATIOS_DES5M.npy', allow_pickle=True).item()
MULTIPOLES = np.load('MULTIPOLES_DES5M.npy', allow_pickle=True).item()
DATA = np.load('DES5M.npy', allow_pickle=True).item()

In [5]:
alpha_scaler = Alpha(n_steps=2, load_weights=False)
alpha_scaler.load_weights('../source/weights_models/ALPHA2R')
ALPHAS = np.load('ALPHAS_2R.npy', allow_pickle=True).item()
FOLDER = ''

In [6]:
LR_TERMS, FIELDS, MU_IND, POL_TERMS = {}, {}, {}, {}
for idk, key in enumerate(DATA):
    coords_1, coords_2, distance_matrices, elements_1, elements_2, graph_1, graph_2, alphas_1, alphas_2, multipoles = get_batch(key)
    V_esp, F1, F2 = Electrostatics.esp_dimer(coords_1, coords_2, distance_matrices, multipoles, with_field=True)   
    V_pol, mu_ind_1, mu_ind_2 = Polarization.ind_dimer0(alphas_1, alphas_2, F1, F2)
    MU_IND[key] = (mu_ind_1, mu_ind_2)
    LR_TERMS[key] = (V_esp, V_pol)
np.save(f'{FOLDER}MU_IND0', MU_IND)
np.save(f'{FOLDER}LR_TERMS_IND0', LR_TERMS)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
len(MU_IND), len(LR_TERMS)

(118939, 118939)